In [1]:
import json
import os


# test the connection 

IP = "0.0.0.0"
PORT = 23422
os.system('python3 rx.py ' + "\'" + json.dumps({'IP': IP, 'PORT': PORT}) + "\'")


Corrupt JPEG data: premature end of data segment
2024-11-10 14:36:20.468 Python[7604:161334] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


avg_verification: 0.9988095238095239, latency: -0.0018962415372601083, goodput: 0.30449396813702917
3 fps


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: 877 extraneous bytes before marker 0xd9


9 fps


Corrupt JPEG data: 52 extraneous bytes before marker 0xd9
Corrupt JPEG data: 22 extraneous bytes before marker 0xda


...

7 fps


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 770 extraneous bytes before marker 0xd9
Corrupt JPEG data: 552 extraneous bytes before marker 0xd9


4 fps
Creating a new file


Corrupt JPEG data: 349 extraneous bytes before marker 0xd9


0

In [2]:
#### parameters that needs to be exhanged between the sender and the receiver #####
IP = "0.0.0.0"
PORT = 23422
#################################################################################### 

while True:
   os.system('python3 rx.py ' + "\'" + json.dumps({'IP': IP, 'PORT': PORT}) + "\'")


2024-11-10 14:36:52.822 Python[7610:161640] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


avg_verification: 1.0, latency: -0.025859908650561075, goodput: 0.5672632921412551
3 fps
8 fps
8 fps
5 fps
6 fps
avg_verification: 1.0, latency: -0.02691696910512007, goodput: 0.5619850326285786
5 fps
6 fps
6 fps
8 fps
8 fps
avg_verification: 1.0, latency: -0.026890079968876098, goodput: 0.562263704378234
8 fps
6 fps
7 fps
6 fps
8 fps
avg_verification: 1.0, latency: -0.026916167216225834, goodput: 0.5624795248701296
8 fps
7 fps
7 fps
10 fps
10 fps
avg_verification: 1.0, latency: -0.02701599004667267, goodput: 0.5631441268612188
8 fps
8 fps
